### Feature selection table results per timegroup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import psycopg2.extras
import csv
import re

In [2]:
def get_data_regularity():
    
    #SQL statement regularity_trip_timegroups_results
    sql = """select *
from regularity_trip_timegroups_results
order by route_short_name, stop_id, direction_id, from_date_time"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [3]:
def get_defined_intervals():
    
    #SQL statement regularity_trip_timegroups_results
    sql = """select *
from defined_intervals
order by from_time"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [4]:
data=get_data_regularity()
data=data.dropna()

In [5]:
intervals=get_defined_intervals()

In [6]:
intervals.head()

,from_time,to_time
0,2021-09-01 00:00:00,2021-09-01 00:30:00
1,2021-09-01 00:30:00,2021-09-01 01:00:00
2,2021-09-01 01:00:00,2021-09-01 01:30:00
3,2021-09-01 01:30:00,2021-09-01 02:00:00
4,2021-09-01 02:00:00,2021-09-01 02:30:00


In [7]:
intervals["data"]=None

In [8]:
data.head()

,route_id,route_short_name,stop_id,direction_id,from_date_time,to_date_time,from_time,to_time,from_date,to_date,...,thursday,friday,saturday,sunday,route_type,waiting_time_real,waiting_time_scheduled,ewt,ewt_minutes,ewt_category
5,2,1,8011,1,2021-09-01 05:48:57,2021-09-17 20:24:07,05:48:57,20:24:07,2021-09-01,2021-09-17,...,1,1,0,0,1,232.871609,39.160125,193.711484,3.228525,0-4MIN
6,2,1,8011,1,2021-09-01 20:34:07,2021-09-17 23:48:53,20:34:07,23:48:53,2021-09-01,2021-09-17,...,1,1,0,0,1,410.917755,145.023148,265.894607,4.431577,4-8MIN
7,2,1,8011,1,2021-09-04 06:44:23,2021-09-18 23:48:53,06:44:23,23:48:53,2021-09-04,2021-09-18,...,0,0,1,0,1,259.528527,57.318244,202.210283,3.370171,0-4MIN
8,2,1,8011,1,2021-09-12 08:42:04,2021-09-12 23:48:53,08:42:04,23:48:53,2021-09-12,2021-09-12,...,0,0,1,0,1,286.690589,54.203186,232.487403,3.874790,0-4MIN
9,2,1,8011,1,2021-09-19 06:44:23,2021-09-19 23:48:53,06:44:23,23:48:53,2021-09-19,2021-09-19,...,0,0,1,0,1,308.359592,43.434227,264.925365,4.415423,4-8MIN


In [9]:
intervals.head()

,from_time,to_time,data
0,2021-09-01 00:00:00,2021-09-01 00:30:00,None
1,2021-09-01 00:30:00,2021-09-01 01:00:00,None
2,2021-09-01 01:00:00,2021-09-01 01:30:00,None
3,2021-09-01 01:30:00,2021-09-01 02:00:00,None
4,2021-09-01 02:00:00,2021-09-01 02:30:00,None


In [10]:
data["ewt"]= data["ewt"].astype(int)


In [11]:
data["from_date_time"]= pd.to_datetime(data["from_date_time"])
data["to_date_time"]= pd.to_datetime(data["to_date_time"])

In [12]:
data=data[data["route_id"]==56]

In [13]:
intervals_data=[]
for i in range(0, len(intervals)):
    c_interval=intervals.iloc[i].to_dict()
    temp=[]
    find_elements_range=data[(data["from_date_time"].dt.time<=c_interval["from_time"].time())&(c_interval["to_time"].time()<=data["to_date_time"].dt.time)&(data["from_date_time"].dt.date<=c_interval["from_time"].date())&(c_interval["to_time"].date()<=data["to_date_time"].dt.date)&(data["ewt"]>(6*60))]
    data_text=None
    if(len(find_elements_range)>0):
        temp=find_elements_range["stop_id"].unique()
        data_text=' '.join(str(item) for item in temp)
    intervals_data.append(data_text)
intervals["data"]=intervals_data

In [14]:
intervals=intervals.dropna()

In [15]:
intervals["from_date"]=intervals["from_time"].dt.date
intervals_data_final=intervals.groupby("from_date")["data"].apply(list).reset_index(name='pattern')

In [19]:
intervals_data_final.iloc[1].to_dict()

{'from_date': datetime.date(2021, 9, 2),
 'pattern': ['1030 2088 2237 2238 2239 2269 2271 2587 2831 2898 6459']}

In [20]:
intervals_data_final.iloc[2].to_dict()

{'from_date': datetime.date(2021, 9, 3),
 'pattern': ['1030 2088 2089 2237 2238 2239 2269 2271 2587 5740 5741 5742 5906 5910 6123 6459 6461 9754 9755',
  '1030 1083 2088 2089 2207 2237 2238 2239 2269 2271 2587 2831 5740 5741 5742 5906 5910 6123 6459 6461 9025 9754 9755',
  '1030 1083 2088 2089 2207 2237 2238 2239 2269 2271 2587 2831 5740 5741 5742 5906 5910 6123 6459 6461 9025 9754 9755',
  '1030 1083 2088 2089 2207 2237 2238 2239 2269 2271 2587 2831 5740 5741 5742 5906 5910 6123 6459 6461 9025 9754 9755',
  '1030 1083 2088 2089 2207 2237 2238 2239 2269 2271 2587 2831 5740 5741 5742 5906 5910 6123 6459 6461 9025 9754 9755',
  '1030 1083 2088 2089 2207 2237 2238 2239 2269 2271 2587 2831 5740 5741 5742 5906 5910 6123 6459 6461 9025 9754 9755',
  '1030 1083 2088 2237 2238 2239 2269 2271 2587 2831 6459 9025',
  '2089 2207 5740 5741 5742 5905 5906 5908 5913 6123 6461 9754 9755 9756',
  '1083 1904 2089 2207 2831 5740 5741 5742 5905 5906 5908 5913 6123 6461 9025 9754 9755 9756',
  '1083 1904 

In [22]:
final_data=[]
with open('./data.txt', 'w') as f:
    for i in range(0, len(intervals_data_final)):
        c_interval=intervals_data_final.iloc[i].to_dict()
        list_data=c_interval["pattern"]
        linea=' -1 '.join(str(item) for item in list_data)
        linea=linea+" -2"
        f.write(linea)
        f.write('\n')
      #np.array(intervals_data_final["pattern"][i]).tostring()

In [35]:
!pip3 install spmf --user

In [1]:
import os
import spmf
path = os.path.abspath(spmf.__file__)

In [2]:
path

'/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/spmf/__init__.py'

In [24]:
from spmf import Spmf

spmf= Spmf("PrefixSpan_with_strings", input_filename="./data.txt",output_filename="./output.txt", arguments=[0.7])
spmf.run()
spmf.to_csv("./output.csv")

java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:349)
	at java.base/sun.nio.cs.StreamEncoder.writeBytes(StreamEncoder.java:234)
	at java.base/sun.nio.cs.StreamEncoder.implWrite(StreamEncoder.java:304)
	at java.base/sun.nio.cs.StreamEncoder.implWrite(StreamEncoder.java:282)
	at java.base/sun.nio.cs.StreamEncoder.write(StreamEncoder.java:132)
	at java.base/java.io.OutputStreamWriter.write(OutputStreamWriter.java:205)
	at java.base/java.io.BufferedWriter.flushBuffer(BufferedWriter.java:120)
	at java.base/java.io.BufferedWriter.write(BufferedWriter.java:233)
	at java.base/java.io.Writer.write(Writer.java:249)
	at ca.pfv.spmf.algorithms.sequentialpatterns.BIDE_and_prefixspan_with_strings.AlgoPrefixSpan_with_Strings.savePattern(AlgoPrefixSpan_with_Strings.java:210)
	at ca.pfv.spmf.algorithms.sequentialpatterns.BIDE_and_prefixspan_with_strings.AlgoPrefixSpan_with_Str

>/Users/lauraforerocamacho/Library/Python/3.8/lib/python/site-packages/spmf/spmf.jar
An error while trying to run the algorithm. 
 ERROR MESSAGE = java.io.IOException: No space left on device



: 

: 